In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
from collections import Counter
import math
from PIL import Image
from pathlib import Path


In [ ]:
batch_size = 16
max_epoch = 40
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
train_data_path = '../../Datasets/Model/train'
test_data_path = '../../Datasets/Model/test'
image_path = '../../Datasets/Images'
data_save_path = '../../Save/Experiment2'
analytics_path = '../../Analysis/Experiment2'
lr = 0.0005
rmse_arr = []
tokens = []
with open('../../Datasets/Model/nlp/tokens.txt', 'r') as f:
  for line in f:
    tokens.append(line.strip())


In [ ]:
train_users = np.load(f'{train_data_path}/train_users.npy')
train_items = np.load(f'{train_data_path}/train_items.npy')
train_ratings = np.load(f'{train_data_path}/train_ratings.npy')
train_reviews = np.load(
    f'{train_data_path}/train_reviews.npy', allow_pickle=True)
train_descriptions = np.load(
    f'{train_data_path}/train_descriptions.npy', allow_pickle=True)
train_prices = np.load(f'{train_data_path}/train_prices.npy')
train_categories = np.load(f'{train_data_path}/train_categories.npy')

test_users = np.load(f'{test_data_path}/test_users.npy')
test_items = np.load(f'{test_data_path}/test_items.npy')
test_ratings = np.load(f'{test_data_path}/test_ratings.npy')
test_reviews = np.load(f'{test_data_path}/test_reviews.npy', allow_pickle=True)
test_descriptions = np.load(
    f'{test_data_path}/test_descriptions.npy', allow_pickle=True)
test_prices = np.load(f'{test_data_path}/test_prices.npy')
test_categories = np.load(f'{test_data_path}/test_categories.npy')


In [ ]:
def cycle(iterable):
  while True:
    for x in iterable:
      yield x


train_iterator = cycle(DataLoader(
    np.unique(train_users), batch_size=batch_size, shuffle=True))

test_iterator = cycle(DataLoader(
    np.unique(test_users), batch_size=batch_size, shuffle=True
))


In [ ]:
'''
  input: user u, sets
  item: i
  output: user_reviews <- [reviews of u],  
          user[descriptions that u has seen], 
          most liked category of u, 
          average price of u purchasing items, 
          item_reviews <- [reviews of i],
          item_description <- description of i,
          item_price <- price of i,
          item_image <- image of i,
          unseen_image <- the image of an unseen item to u,
          rev_ui <- rating of u on i

  status: this function works fine
'''


'''vectorise code'''


def sample_user(user, user_set, item_set, rev_set, desc_set, cat_set, price_set, img_set, rating_set):
  # -> u
  user = int(user)
  user_meta = []
  item_meta = []
  user_reviews = []
  user_descriptions = []
  user_category = []
  user_price = []
  item_reviews = []
  item_description = None
  item_category = None
  item_price = None
  item_image = None
  unseen_image = None
  rev_ui = None

  user_indicies = np.where(user_set == user)[0]

  # generates the index of a rated item for the user
  chosen_user_item_idx = user_indicies[random.randint(0, len(user_indicies)-1)]

  # randomly select a rated item -> i
  item = item_set[chosen_user_item_idx]
  item_description = list(desc_set[chosen_user_item_idx])
  item_category = cat_set[chosen_user_item_idx]
  item_price = price_set[chosen_user_item_idx]

  # gets the rating of u, i
  rating_ui = rating_set[chosen_user_item_idx]

  # gets item indices to get all reviews of the item -> i
  item_indicies = np.where(item_set == item)[0]

  # all indicies in arrays are same (identical to user review number)
  for u_idx in user_indicies:
      # out of reviews, other metadata is always accessible either or not in test mode
      user_descriptions += list(desc_set[u_idx])
      user_category.append(cat_set[u_idx])
      user_price.append(price_set[u_idx].astype(np.float32))
      if u_idx != chosen_user_item_idx:
        user_reviews += list(rev_set[u_idx])
      else:
        rev_ui = list(rev_set[u_idx])

  for i_idx in item_indicies:
    if i_idx != chosen_user_item_idx:
      item_reviews += list(rev_set[i_idx])

  # according to the transnets paper - regularise the combined strings
  if len(rev_ui) > 1000:
    rev_ui = rev_ui[:1000]
  if len(user_reviews) > 1000:
    user_reviews = user_reviews[:1000]
  if len(user_descriptions) > 1000:
    user_descriptions = user_descriptions[:1000]
  if len(item_reviews) > 1000:
    item_reviews = item_reviews[:1000]

  if len(rev_ui) <= 3:
    rev_ui += [0] * 3
  if len(user_reviews) <= 3:
    user_reviews += [0] * 3
  if len(item_reviews) <= 3:
    item_reviews += [0] * 3
  if len(user_descriptions) <= 3:
    user_descriptions += [0] * 3

  # for computational correctness
  user_reviews, user_descriptions, item_reviews, item_description = torch.tensor(user_reviews), torch.tensor(user_descriptions),\
      torch.tensor(item_reviews), torch.tensor(item_description)

  user_category = Counter(user_category).most_common(1)[0][0]
  user_price = sum(user_price)/len(user_price)

  rev_ui = torch.from_numpy(np.array(rev_ui))

  user_meta = [torch.tensor(user_category), torch.tensor(
      [user_price], dtype=torch.float32), user_descriptions, user_reviews]
  item_meta = [torch.tensor(item), torch.tensor(item_category), torch.tensor(
      [item_price], dtype=torch.float32), item_description, item_reviews]

  return user_meta, item_meta, item_image, unseen_image, rev_ui, torch.tensor(rating_ui, dtype=torch.float32)


def weights_init(m):
  if type(m) in (nn.Linear, nn.Conv1d, nn.Conv2d, nn.Embedding):
    nn.init.xavier_normal_(m.weight)


In [ ]:
class FM(nn.Module):
    def __init__(self, latent_dim, fea_num):
        super().__init__()

        self.latent_dim = latent_dim
        self.w0 = nn.Parameter(torch.zeros([1, ]))
        self.w1 = nn.Parameter(torch.rand([fea_num, 1]))
        self.w2 = nn.Parameter(torch.rand([fea_num, latent_dim]))

    def forward(self, inputs):
        # inputs = inputs.long()
        first_order = self.w0 + torch.mm(inputs, self.w1)
        second_order = 1/2 * torch.sum(
            torch.pow(torch.mm(inputs, self.w2), 2) -
            torch.mm(torch.pow(inputs, 2), torch.pow(self.w2, 2)),

            dim=1,
            keepdim=True
        )

        return first_order + second_order


class GMF(nn.Module):
    def __init__(self, inp_range, latent_dim=20, dropout=True):
        super().__init__()
        self.emb = nn.Embedding(
            num_embeddings=inp_range, embedding_dim=latent_dim)
        self.dropout = nn.Dropout(0.5)
        self.use_dropout = dropout

    def forward(self, inputs):
        embedding = self.emb(inputs)
        if self.use_dropout:
          embedding = self.dropout(embedding)

        return embedding


fm_S = FM(8, 50).apply(weights_init).to(device)

mf_u = GMF(max(train_users)+1, 25, True).apply(weights_init).to(device)
# item latent vectors -> this embedding should be updated
mf_i = GMF(max(max(train_items), max(test_items))+1,
           25, True).apply(weights_init).to(device)

optimiser_MFU = torch.optim.Adam(
    params=mf_u.parameters(), lr=lr)
optimiser_MFI = torch.optim.Adam(
    params=mf_i.parameters(), lr=lr)

optimiser_FMS = torch.optim.Adam(
    params=fm_S.parameters(), lr=lr, weight_decay=0.01)


def save_training():
  torch.save(
      {
       'fmS': fm_S.state_dict(),
       'mf_u': mf_u.state_dict(),
       'mf_i': mf_i.state_dict()},
      f'{data_save_path}/gmf.chkpt')



In [ ]:
def eval_rmse(preds, real):
    rmse = 0
    # produce known rui set
    for i in range(len(preds)):
        rmse += (preds[i] - real[i]) ** 2

    rmse = math.sqrt(rmse/len(preds))

    return rmse


def l1regularise(mode):
  global textCNN_I
  global textCNN_U
  global textCNN_T
  global transform
  global fm_T
  global fm_S

  model = None

  if mode == 'I':
    model = textCNN_I
  elif mode == 'U':
    model = textCNN_U
  elif mode == 'T':
    model = textCNN_T
  elif mode == 'transform':
    model = transform
  elif mode == 'fmt':
    model = fm_T
  else:
    model = fm_S

  reg_loss = 0
  for param in model.parameters():
    reg_loss += torch.sum(torch.abs(param))

  return reg_loss


def eval_model():
  preds = []
  real = []
  for u in tqdm(np.unique(test_users)):
    user_meta, item_meta, _, _, _, rating_ui = sample_user(
        u, test_users, test_items, test_reviews, test_descriptions, test_categories, test_prices, None, test_ratings)
    _, _, _, user_reviews = user_meta
    i, _, _, _, item_reviews = item_meta
    user_reviews = user_reviews.to(device)
    item_reviews = item_reviews.to(device)
    i = i.to(device)

    '''User and Item'''
    latent_uid = mf_u(torch.tensor(u).to(device))
    latent_iid = mf_i(i)  # -> 20

    latent_final = torch.cat(
        (
         latent_uid,  # 10
         latent_iid  # 10
         ),  # 1
        dim=0).view(1, 50)

    pred = fm_S(latent_final)
    preds.append(pred)
    real.append(rating_ui)

  return eval_rmse(preds, real)


In [ ]:
print("evaluating baseline rmse, please wait...")

base_rmse = eval_model()

# start training
print(f"training started, baseline rmse:{base_rmse}")

best_trans_params = None
best_source_params = None
best_rmse = np.inf

l1_loss = nn.L1Loss(reduction='mean')
l2_loss = nn.MSELoss(reduction='mean')

for epoch in range(0, max_epoch):
  for i in tqdm(range(1000)):
    # batch_size = 50
    users = next(train_iterator)
    users = users.to(device)
    loss_S = 0

    for u in users:
      '''Get data needed'''
      user_meta, item_meta, _, _, rev_ui, rating_ui = sample_user(
          u, train_users, train_items, train_reviews, train_descriptions, train_categories, train_prices, None, train_ratings)
      _, _, _, user_reviews = user_meta
      i, _, _, _, item_reviews = item_meta
      user_reviews = user_reviews.to(device)
      item_reviews = item_reviews.to(device)
      i = i.to(device)

      '''Train a predictor on the transformed input'''

      '''User and Item'''
      latent_uid = mf_u(u)
      latent_iid = mf_i(i)  # -> 20

      latent_final = torch.cat(
          (
           latent_uid,  # 10
           latent_iid  # 10
           ),  # 1
          dim=0).view(1, 50)

      pred_S = fm_S(latent_final)
      loss_S += l1_loss(rating_ui, torch.flatten(pred_S)[0])

    optimiser_MFI.zero_grad()
    optimiser_MFU.zero_grad()
    optimiser_FMS.zero_grad()
    loss_S /= batch_size
    loss_S.backward()
    optimiser_FMS.step()
    optimiser_MFU.step()
    optimiser_MFI.step()

  with torch.no_grad():
    rmse = eval_model()
    rmse_arr.append(rmse)

    if rmse < best_rmse:
      best_rmse = rmse
      save_training()
    print(f"epoch: [{epoch}/{max_epoch}]: rmse - {rmse}")


In [ ]:
save_training()